In [5]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # for visualiation
import scipy
from datetime import datetime
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
import sklearn as sk

pd.options.display.max_columns = None
%matplotlib inline

In [2]:
df = pd.read_csv('./data/shootings_and_sales_per_pop.csv')

In [3]:
df.head()

,year,state,population,admin,handgun,long_gun,month,multiple,other,permit,totals,killed,killed_max,injured,incidents,injured_max,affected_max,affected_over_10,killed_ave,injured_ave,admin_pop,handgun_pop,long_gun_pop,multiple_pop,other_pop,permit_pop,totals_pop,killed_pop,injured_pop,incidents_pop
0,2013,United States,316234505.0,267.0,881038.0,788328.0,1,31029.0,26137.0,693076.0,2483230.0,16.0,5.0,39.0,12.0,6.0,7.0,0.0,1.333333,3.250000,0.084431,278.602741,249.285890,9.812022,8.265069,219.165205,785.249541,0.005060,0.012333,0.003795
1,2013,United States,316234505.0,375.0,770872.0,703223.0,2,31163.0,22709.0,664252.0,2298561.0,14.0,4.0,45.0,12.0,8.0,8.0,0.0,1.166667,3.750000,0.118583,243.765936,222.373899,9.854396,7.181063,210.050450,726.853320,0.004427,0.014230,0.003795
2,2013,United States,316234505.0,376.0,683266.0,700873.0,3,23878.0,23817.0,695386.0,2197116.0,25.0,6.0,75.0,19.0,13.0,13.0,1.0,1.315789,3.947368,0.118899,216.063076,221.630780,7.550726,7.531436,219.895675,694.774278,0.007906,0.023717,0.006008
3,2013,United States,316234505.0,665.0,533674.0,554003.0,4,18624.0,21136.0,517224.0,1702455.0,26.0,6.0,52.0,17.0,5.0,7.0,0.0,1.529412,3.058824,0.210287,168.758940,175.187398,5.889300,6.683648,163.557105,538.352069,0.008222,0.016443,0.005376
4,2013,United States,316234505.0,619.0,445522.0,436674.0,5,16057.0,15605.0,449993.0,1424450.0,18.0,5.0,99.0,24.0,19.0,19.0,1.0,0.750000,4.125000,0.195741,140.883424,138.085501,5.077561,4.934629,142.297249,450.441042,0.005692,0.031306,0.007589


In [14]:
df =df.fillna(0)

In [15]:
training = ['year','population',
            'killed',
            'killed_max','injured','incidents',
            'injured_max','affected_max','affected_over_10',
            'killed_ave','injured_ave',
            'killed_pop','injured_pop','incidents_pop']
testing = ['totals_pop']
# 'admin_pop',
#             'handgun_pop','long_gun_pop','multiple_pop',
#             'other_pop','permit_pop',

In [20]:
rfecv

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
   n_jobs=1, scoring='neg_mean_squared_error', step=1, verbose=0)

In [27]:


# Build a classification task using 3 informative features
X_train, X_test, y_train, y_test  = sk.model_selection.train_test_split(df[training],df[testing],train_size=.8,random_state=10)

# Create the RFE object and compute a cross-validated score.
svr = sk.svm.SVR()
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svr, step=1, cv=StratifiedKFold(5),
              scoring='neg_mean_squared_error')
rfecv.fit(X_train, y_train)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

C:\Users\Elias\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\Elias\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.